#### **Import Libraries**

In [1]:
import os
import shutil
import json
from tqdm.auto import tqdm

c:\Users\LONG\anaconda3\envs\aic2024-env-new\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%cd D:/AIC2024/dataset

D:\AIC2024\dataset


#### **Parsing Data Path**

In [3]:
def parse_data_path(feature_dir='./keyframe'):
    all_feature_paths = dict()
    for feature_part in sorted(os.listdir(feature_dir)):
        all_feature_paths[feature_part] = dict()
    for feature_part in sorted(all_feature_paths.keys()):
        feature_part_path = f'{feature_dir}/{feature_part}'
        feature_paths = sorted(os.listdir(feature_part_path))
        feature_ids = [feature_path.split('.')[0] for feature_path in feature_paths]
        for feature_id, feature_path in zip(feature_ids, feature_paths):
            feature_path_full = f'{feature_part_path}/{feature_path}'
            all_feature_paths[feature_part][feature_id] = feature_path_full
    return all_feature_paths

In [4]:
all_video_paths = parse_data_path(feature_dir='./distillation')

#### **Function Definition**

In [5]:
def copy_images(image_paths, destination_dir):
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)
    for image_path in image_paths:
        image_name = os.path.basename(image_path)
        destination_path = os.path.join(destination_dir, image_name)
        shutil.copy(image_path, destination_path)

def read_json_file(json_file_path):
    with open(json_file_path, 'r') as f:
        image_paths = json.load(f)
    return image_paths

def sorted_by_id(keyframe_paths):
    id_path_keyframes = []
    for keyframe_path in keyframe_paths:
        keyframe_filename = keyframe_path.split('/')[-1]
        keyframe_id = int(keyframe_filename.split('.')[0])
        id_path_keyframes.append((keyframe_id, keyframe_path))
    sorted_id_path_keyframes = sorted(id_path_keyframes, key=lambda id_path: id_path[0])
    return [id_path[1] for id_path in sorted_id_path_keyframes]

#### **Inference**

In [8]:
distilled_save_dir='./distilled_keyframe'
if not os.path.exists(distilled_save_dir):
    os.makedirs(distilled_save_dir)

In [6]:
execute = ['L02', 'L09']

In [9]:
for video_part, video_path_dict in all_video_paths.items():
    video_ids = video_path_dict.keys()
    if video_part not in execute:
        continue
    for video_id in tqdm(video_ids, desc=f'Distilling Part {video_part}'):
        video_id_path = video_path_dict[video_id]
        keyframe_image_paths = read_json_file(video_id_path)
        sorted_keyframe_image_paths = sorted_by_id(keyframe_image_paths)
        destination_dir = f'{distilled_save_dir}/{video_part}/{video_id}'
        os.makedirs(destination_dir, exist_ok=True)
        copy_images(sorted_keyframe_image_paths, destination_dir)

Distilling Part L09: 100%|██████████| 29/29 [05:54<00:00, 12.24s/it]
